# Tipy a triky pro pandas

## Cíle - co se naučíme
* Číst data z 'nehezkého' Excelu
* Indexaci pomocí `loc` a `iloc`
* Odvozovat další sloupce pomocí řádkových funkcí
* Indexaci boolovskými hodnotami
* Vytvářet proměnné z textových proměnných
* Parsovat český formát data `01.12.2020`
* Vytvářet proměnné z datumu a času
* Vytvářet kontingenční tabulky a jiné agegace (včetně nových proměnných)

Nakonec budeme pracovat s reálnými daty z oblasti duplicitních skenů.

Začněme importem knihovny pandas. Vždy takto, je to idiom.

In [1]:
import pandas as pd

## Čtení dat z "nehezkého" Excelu

In [2]:
df = pd.read_excel("/data/duplicity/Duplicity_MobSken-2021_03-05.xlsx", sheet_name="Duplicity-MobSken", skiprows=4,engine='openpyxl')

In [3]:
df.head();

In [4]:
df.head(5).transpose();

## Ze dvou sloupců uděláme jiné dva

In [5]:
def get_origin_and_origin_type(row):
    if pd.isna(row['Pracovník']):
        origin = 'Pobočka'
    else:
        origin = 'Pracovník'
    return pd.Series({'Typ zdroje': origin, 
                      'Zdroj': row[origin]})           

In [6]:
df_origin = df.apply(get_origin_and_origin_type,axis=1)
df = pd.concat([df, df_origin], axis=1)

In [7]:
df.head().transpose();

In [8]:
df.columns;

In [9]:
df.loc[:,['Pojistná smlouva', 'Pojistná událost', 'Číslo návrhu',
       'Obchodní případ',]].notna().sum(axis=1).max();

Závěr: Tyto tři objekty se někdy vyskytují dohromady. Nemá cenu s nimi zacházet podobně jako se sloupci `Pracovník` a `Pobočka`.

## Proměnné z textových sloupců
Jaké máme druhy faktur?



In [10]:
df_faktury = df[df['Typ dokumentu'].notna() & df['Typ dokumentu'].str.contains('aktur')]
df_faktury;

In [11]:
df_faktury['Typ dokumentu'].unique();

In [12]:
df_faktury.groupby('Typ dokumentu').count()['ID dokumentu'];

## Práce s datumem a časem

In [13]:
df['Datum skenování'];

In [14]:
pd.to_datetime(df['Datum skenování']);

In [15]:
df['Datum skenování'] = pd.to_datetime(df['Datum skenování'], format="%d.%m.%Y");

In [16]:
df['Datum skenování'].dt.isocalendar().week;

In [17]:
df['Týden'] = df['Datum skenování'].dt.isocalendar().week;

In [18]:
df['Týden'].unique();

## Kontingenční tabulky

In [19]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['axes.facecolor'] = 'gray'
plt.rcParams['figure.figsize'] = (20,5)

In [20]:
def pivot_table(df, columns):
    return df.groupby(columns).count()['ID dokumentu'].unstack().fillna(0)

pivot_week = pivot_table(df, ['Týden', 'Barva'])



In [21]:
#pivot_week.plot.bar(color=pivot_week.columns);

In [22]:
#(pivot_week['Blue']/pivot_week['Green']).plot.bar()

In [23]:
pivot_typ_zdroje = pivot_table(df, ['Typ zdroje', 'Barva'])
pivot_typ_zdroje;

In [24]:
#pivot_typ_zdroje.plot.bar(color=pivot_typ_zdroje.columns);

In [25]:
pivot_typ_zdroje['Blue']/pivot_typ_zdroje['Green'];

# #############